Next Word Prediction Model - using RNN and LSTM.

Installing Required Libraries and Packges

In [1]:
import numpy as np 
import pandas as pd  
import tensorflow as tf
from colorama import Back, Fore
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from termcolor import colored, cprint

Dataset Information : Medium Articles Dataset
    Link: https://www.kaggle.com/datasets/dorianlazar/medium-articles-dataset?select=medium_data.csv


In [2]:
df = pd.read_csv("medium_data.csv")
df.head()

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30
1,2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,NaN,2.png,1100,11,9,Towards Data Science,2019-05-30
2,3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,1,5,Towards Data Science,2019-05-30
3,4,https://towardsdatascience.com/databricks-how-...,Databricks: How to Save Files in CSV on Your L...,When I work on Python projects dealing…,4.jpeg,354,0,4,Towards Data Science,2019-05-30
4,5,https://towardsdatascience.com/a-step-by-step-...,A Step-by-Step Implementation of Gradient Desc...,One example of building neural…,5.jpeg,211,3,4,Towards Data Science,2019-05-30


check shape of data

In [3]:
print("Number of records: ", df.shape[0])
print("Number of fields: ", df.shape[1])

Number of records:  6508
Number of fields:  10


Display titles of various articles and preprocess them

In [4]:
df["title"]

0       A Beginner’s Guide to Word Embedding with Gens...
1       Hands-on Graph Neural Networks with PyTorch & ...
2                            How to Use ggplot2 in Python
3       Databricks: How to Save Files in CSV on Your L...
4       A Step-by-Step Implementation of Gradient Desc...
                              ...                        
6503    “We” vs “I” — How Should You Talk About Yourse...
6504                     How Donald Trump Markets Himself
6505        Content and Marketing Beyond Mass Consumption
6506    5 Questions All Copywriters Should Ask Clients...
6507               How To Write a Good Business Blog Post
Name: title, Length: 6508, dtype: object

In [5]:
df["title"].isna().sum()

0

Data cleaning 

In [6]:
df["title"] = df["title"].apply(lambda x: x.replace("\xa0", " "))
df["title"] = df["title"].apply(lambda x: x.replace("\u200a", " "))

In [7]:
df["title"]

0       A Beginner’s Guide to Word Embedding with Gens...
1       Hands-on Graph Neural Networks with PyTorch & ...
2                            How to Use ggplot2 in Python
3       Databricks: How to Save Files in CSV on Your L...
4       A Step-by-Step Implementation of Gradient Desc...
                              ...                        
6503    “We” vs “I” — How Should You Talk About Yourse...
6504                     How Donald Trump Markets Himself
6505        Content and Marketing Beyond Mass Consumption
6506    5 Questions All Copywriters Should Ask Clients...
6507               How To Write a Good Business Blog Post
Name: title, Length: 6508, dtype: object

Tokenization

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [9]:
tokenizer = Tokenizer(
    oov_token="<oov>"
)  # For those words which are not found in word_index
tokenizer.fit_on_texts(df["title"])
total_words = len(tokenizer.word_index) + 1

print("Total number of words: ", total_words)
print("Word: ID")
print("------------")
print("<oov>: ", tokenizer.word_index["<oov>"])
print("Strong: ", tokenizer.word_index["strong"])
print("And: ", tokenizer.word_index["and"])
print("Consumption: ", tokenizer.word_index["consumption"])
print("------------")
print("Example how the data is stored :-\n", list(tokenizer.word_index.items())[:5])

Total number of words:  8238
Word: ID
------------
<oov>:  1
Strong:  4
And:  8
Consumption:  8237
------------
Example how the data is stored :-
 [('<oov>', 1), ('to', 2), ('the', 3), ('strong', 4), ('a', 5)]


In [10]:
input_sequences = []
for line in df["title"]:
    token_list = tokenizer.texts_to_sequences([line])[0]
  
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[: i + 1]
        input_sequences.append(n_gram_sequence)

print("This is What token list looks like :-", token_list)
cprint("------------", "yellow")
# print(input_sequences)
print("Total input sequences: ", len(input_sequences))
cprint("------------", "yellow")
print(list(tokenizer.word_index.items())[4])
print(list(tokenizer.word_index.items())[675])
print(list(tokenizer.word_index.items())[67])
print(list(tokenizer.word_index.items())[1])
print(list(tokenizer.word_index.items())[451])
print(list(tokenizer.word_index.items())[1517])
cprint("------------", "yellow")
print("This sentence is converted into the below sequence ::")
print(Back.RED, df["title"][0])
# print(len(list(tokenizer.word_index.items())))
input_sequences[:9]

This is What token list looks like :- [7, 2, 65, 5, 85, 56, 730, 550]
------------
Total input sequences:  48461
------------
('a', 5)
('beginner’s', 676)
('guide', 68)
('to', 2)
('word', 452)
('embedding', 1518)
------------
This sentence is converted into the below sequence ::
 A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model


[[5, 676],
 [5, 676, 68],
 [5, 676, 68, 2],
 [5, 676, 68, 2, 452],
 [5, 676, 68, 2, 452, 1518],
 [5, 676, 68, 2, 452, 1518, 14],
 [5, 676, 68, 2, 452, 1518, 14, 2455],
 [5, 676, 68, 2, 452, 1518, 14, 2455, 3653],
 [5, 676, 68, 2, 452, 1518, 14, 2455, 3653, 99]]

In [11]:
token_list

[7, 2, 65, 5, 85, 56, 730, 550]

In [12]:
import pickle

with open("tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


Make all titles with same length by using padding

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_len = max([len(x) for x in input_sequences])
# get the maximum length sequence
max_sequence_len

40

In [14]:
input_sequences = np.array(
    pad_sequences(input_sequences, maxlen=max_sequence_len, padding="pre")
)
input_sequences[8]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    5,  676,   68,
          2,  452, 1518,   14, 2455, 3653,   99])


Prepare features and labels

In [15]:
input_sequences[:, :]  # This is the original sequences

array([[  0,   0,   0, ...,   0,   5, 676],
       [  0,   0,   0, ...,   5, 676,  68],
       [  0,   0,   0, ..., 676,  68,   2],
       ...,
       [  0,   0,   0, ...,   5,  85,  56],
       [  0,   0,   0, ...,  85,  56, 730],
       [  0,   0,   0, ...,  56, 730, 550]])

In [16]:
input_sequences[:, :-1] # This is the sequences leaving the last element

array([[  0,   0,   0, ...,   0,   0,   5],
       [  0,   0,   0, ...,   0,   5, 676],
       [  0,   0,   0, ...,   5, 676,  68],
       ...,
       [  0,   0,   0, ...,  65,   5,  85],
       [  0,   0,   0, ...,   5,  85,  56],
       [  0,   0,   0, ...,  85,  56, 730]])

In [17]:
input_sequences[:, -1]  # This is the last element which was omitted

array([676,  68,   2, ...,  56, 730, 550])

In [18]:
xs, labels = input_sequences[:, :-1], input_sequences[:, -1]
print(xs[5])
print(labels[5])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    5  676   68    2  452 1518]
14


In [19]:
cprint("Performing ONE-HOT ENCODING.....", "green")
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)
print("At row-5 and col-14 ==", ys[5][14])
pd.DataFrame(ys).head(6)

Performing ONE-HOT ENCODING.....
At row-5 and col-14 == 1.0


,0,1,2,3,4,5,6,7,8,9,...,8228,8229,8230,8231,8232,8233,8234,8235,8236,8237
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


RNN and Bidirectional LSTM Neural Network

In [20]:
# Build the model
model = Sequential([
    Embedding(total_words, 100),  
    Bidirectional(LSTM(150)),
    Dense(total_words, activation="softmax")
])

# Compile the model
adam = Adam(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

# Train the model
history = model.fit(xs, ys, epochs=50, verbose=1)

# Print model summary
model.summary()


Epoch 1/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 96s 60ms/step - accuracy: 0.0991 - loss: 7.0921
Epoch 2/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 99s 65ms/step - accuracy: 0.1542 - loss: 6.9647
Epoch 3/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 97s 64ms/step - accuracy: 0.1884 - loss: 5.2439
Epoch 4/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 90s 59ms/step - accuracy: 0.2342 - loss: 4.4013
Epoch 5/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 89s 59ms/step - accuracy: 0.2914 - loss: 3.8643
Epoch 6/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 89s 59ms/step - accuracy: 0.3337 - loss: 3.3831
Epoch 7/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 89s 59ms/step - accuracy: 0.3677 - loss: 3.1112
Epoch 8/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 139s 57ms/step - accuracy: 0.3831 - loss: 3.0293
Epoch 9/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 88s 58ms/step - accuracy: 0.3963 - loss: 2.9576
Epoch 10/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 89s 58ms/step - accuracy: 0.4029 - loss: 2.9289
Epoch 11/50
1515/1515 ━━━━━━━━━━━━━━━━━━━━ 89s 59ms/step - accuracy: 0.4258 - loss: 2.7549
Epoch 1

KeyboardInterrupt: 

Plotting model accuracy and loss 

In [21]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.show()

Graph of Accuracy

In [22]:
plot_graphs(history, "accuracy")

NameError: name 'history' is not defined

Graph of Loss

In [ ]:
plot_graphs(history, "loss")

In [ ]:
def output(text):
    seed_text = text
    next_words = 5

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences(
            [token_list], maxlen=max_sequence_len - 1, padding="pre"
        )
        predicted = np.argmax(model.predict(token_list), axis=-1)
        # model.predict_classes(token_list, verbose=0)
        cprint(predicted, "red")
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text)


output("implementation of")

In [51]:
output("python for")

NameError: name 'output' is not defined

In [ ]:
model.save("my_model.h5")



In [ ]:
new_model = tf.keras.models.load_model("my_model.h5")
new_model.summary()

In [ ]:
def output_2(text):
    seed_text = text
    next_words = 5

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        print(token_list)
        token_list = pad_sequences(
            [token_list], maxlen=max_sequence_len - 1, padding="pre"
        )

        predicted = np.argmax(new_model.predict(token_list), axis=-1)
        # model.predict_classes(token_list, verbose=0)
        cprint(predicted, "red")
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text)


output_2("implementation")